In [1]:
!pip install textattack[tensorflow,optional]
!pip install -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 152.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 147.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 56.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 205.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 194.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

2024-07-21 13:23:27.255653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 13:23:27.282541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 13:23:27.290051: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 13:23:27.316907: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
textattack: Updating TextAttack package dependencies.

2024-07-21 13:23:31 INFO: Downloaded file to /root/stanza_resources/resources.json
2024-07-21 13:23:31 INFO: Downloading default packages for language: en (English) ...


2024-07-21 13:23:33 INFO: Downloaded file to /root/stanza_resources/en/default.zip
2024-07-21 13:23:36 INFO: Finished downloading models and saved to /root/stanza_resources


### Clean counter-fitted vectors

In [3]:
# read counter-fitted vectors
df = pd.read_table('counter-fitted-vectors.txt', header=None, sep=' ')
df.set_index(0, inplace=True)
df.shape

(65713, 300)

In [4]:
# check any non-string index
[idx for idx, w in enumerate(df.index) if type(w) != type('word')]

[6167, 55574]

In [5]:
# cleaned
new_index = list(df.index)
new_index[6167] = 'null'
new_index[55574] = 'nan'
df.index = new_index
[idx for idx, w in enumerate(df.index) if type(w) != type('word')]

[]

### BERT-based-uncased

In [6]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
# obtain BERT encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/bertvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'w') as f:
    for vocab in new_index[0:25000]:
        cnt += 1
        t = bert_tokenizer(vocab, return_tensors="pt", padding=True, truncation=True)
        t = bert_model(**t).pooler_output.detach().numpy()

        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

.........................2:25:23.768594


In [11]:
# obtain BERT encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/bertvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'a') as f:
    for vocab in new_index[25000:50000]:
        cnt += 1
        t = bert_tokenizer(vocab, return_tensors="pt", padding=True, truncation=True)
        t = bert_model(**t).pooler_output.detach().numpy()

        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

.........................2:34:11.612808


In [8]:
# obtain BERT encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/bertvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'a') as f:
    for vocab in new_index[50000:]:
        cnt += 1
        t = bert_tokenizer(vocab, return_tensors="pt", padding=True, truncation=True)
        t = bert_model(**t).pooler_output.detach().numpy()

        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

...............1:36:30.176722


In [16]:
pd.read_csv('FM_embeddings/bertvocab.txt', header=None).shape

(65713, 768)

### DistilBERT

In [ ]:
# disbert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

In [7]:
# obtain DistilBERT encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/disbertvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'w') as f:
    for vocab in new_index[0:25000]:
        cnt += 1
        t = disbert_tokenizer(vocab, return_tensors="pt", padding=True, truncation=True)
        t = disbert_model(**t).last_hidden_state.detach().numpy().mean(axis=1)
        
        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

.........................1:30:29.612975


In [ ]:
# obtain DistilBERT encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/disbertvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'a') as f:
    for vocab in new_index[25000:50000]:
        cnt += 1
        t = disbert_tokenizer(vocab, return_tensors="pt", padding=True, truncation=True)
        t = disbert_model(**t).last_hidden_state.detach().numpy().mean(axis=1)
        
        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

.............

In [22]:
pd.read_csv('FM_embeddings/disbertvocab.txt', header=None).tail(2)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
42918,-0.110487,-0.067418,-0.237166,-0.105741,0.057214,-0.056128,0.189715,0.331307,-0.105711,-0.170788,...,0.167579,-0.132384,0.064212,-0.534442,-0.058092,-0.422551,-0.095220,0.129420,0.112832,0.180244
42919,0.258264,-0.255038,-0.107135,0.092623,0.224545,0.023881,0.142203,0.038446,0.161466,0.051100,...,0.101525,-0.191026,0.009541,-0.239932,0.355581,-0.390902,-0.169831,-0.216361,0.059144,-0.338457


In [23]:
pd.read_csv('FM_embeddings/disbertvocab.txt', header=None).shape

(42920, 768)

In [24]:
# kernel dies down previous cell at 42920 entries
# obtain DistilBERT encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/disbertvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'a') as f:
    for vocab in new_index[42920:]:
        cnt += 1
        t = disbert_tokenizer(vocab, return_tensors="pt", padding=True, truncation=True)
        t = disbert_model(**t).last_hidden_state.detach().numpy().mean(axis=1)
        
        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

......................1:21:14.731595


In [25]:
pd.read_csv('FM_embeddings/disbertvocab.txt', header=None).shape

(65713, 768)

### CLIP

In [6]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [7]:
# obtain clip encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/clipvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'w') as f:
    for vocab in new_index[0:25000]:
        cnt += 1
        t = clip_processor(vocab, return_tensors="pt", padding=True, truncation=True)
        t = text_encoder(**t).pooler_output.detach().numpy()
        
        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

.........................1:49:21.316979


In [ ]:
# kernel dies down previous cell at 42920 entries
# obtain clip encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/clipvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'a') as f:
    for vocab in new_index[25000:50000]:
        cnt += 1
        t = clip_processor(vocab, return_tensors="pt", padding=True, truncation=True)
        t = text_encoder(**t).pooler_output.detach().numpy()
        
        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

..............

In [8]:
pd.read_csv('FM_embeddings/clipvocab.txt', header=None).shape

(39977, 512)

In [9]:
# kernel dies down previous cell at 42920 entries
# obtain clip encoding
np.set_printoptions(precision=6)
cnt = 0
path = 'FM_embeddings/clipvocab.txt'      # path

st = datetime.datetime.now()

with open(path, 'a') as f:
    for vocab in new_index[39977:]:
        cnt += 1
        t = clip_processor(vocab, return_tensors="pt", padding=True, truncation=True)
        t = text_encoder(**t).pooler_output.detach().numpy()
        
        t = np.round(t[0],6).astype('str')
        for s in t[:-1]:
            f.write(s+',')
        f.write(t[-1]+'\n')
        
        if cnt%1000 == 0:
            print('.',end='')

et = datetime.datetime.now()
print(et-st)

.........................1:46:02.558216
